## 题目

给定一个数组，它的第 i 个元素是一支给定的股票在第 i 天的价格。

设计一个算法来计算你所能获取的最大利润。你最多可以完成 k 笔交易。

注意: 你不能同时参与多笔交易（你必须在再次购买前出售掉之前的股票）。

## 股票问题解析

以下内容主要来自于力扣用户labuladong写的[题解](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-iv/solution/yi-ge-tong-yong-fang-fa-tuan-mie-6-dao-gu-piao-w-5/)。

动态规划两部分：

1. 状态转移框架

首先我们穷举这个问题的所有的状态。

这个问题的「状态」有三个，第一个是天数，第二个是允许交易的最大次数，第三个是当前的持有状态（即之前说的`rest`的状态，我们不妨用 `1`表示持有，
`0`表示没有持有）。然后我们用一个三维数组就可以装下这几种状态的全部组合：

```
dp[i][k][0 or 1]
0 <= i <= n-1, 1 <= k <= K
n 为天数，大 K 为最多交易数
此问题共 n × K × 2 种状态，全部穷举就能搞定。

for 0 <= i < n:
    for 1 <= k <= K:
        for s in {0, 1}:
            dp[i][k][s] = max(buy, sell, rest)
```
`dp[3][2][1]`的含义就是：今天是第三天，我现在手上持有着股票，至今最多进行 `2`次交易；`dp[2][3][0]`的含义：今天是第二天，我现在手上没有持有股票，至今最多进行`3`次交易。

我们想求的最终答案是`dp[n-1][K][0]`，即最后一天，最多允许`K`次交易，最多获得多少利润。为什么不是`dp[n-1][K][1]`？因为`[1]`代表手上还持有股票，`[0]`表示手上的股票已经卖出去了，很显然后者得到的利润一定大于前者。

通过对状态的穷举，得到股票问题的通用状态转移框架：

![股票问题状态转移图](https://pic.leetcode-cn.com/c4eb5f0aa4daf7bef4b3b8af95129bb7394ec58e1ba7b191d9104bbd8ff1ccb3-40198bf2f6894018328b250b772b4a17724a983f99ba359b798a289733bffcbc-file_1559885188422-1.png)

通过这个图可以很清楚地看到，每种状态（0 和 1）是如何转移而来的。根据这个图，状态转移方程如下：

```
dp[i][k][0] = max(dp[i-1][k][0], dp[i-1][k][1] + prices[i])
              max(   选择 rest  ,           选择 sell      )

解释：今天我没有持有股票，有两种可能：
要么是我昨天就没有持有，然后今天选择 rest，所以我今天还是没有持有；
要么是我昨天持有股票，但是今天我 sell 了，所以我今天没有持有股票了。

dp[i][k][1] = max(dp[i-1][k][1], dp[i-1][k-1][0] - prices[i])
              max(   选择 rest  ,           选择 buy         )

解释：今天我持有着股票，有两种可能：
要么我昨天就持有着股票，然后今天选择 rest，所以我今天还持有着股票；
要么我昨天本没有持有，但今天我选择 buy，所以今天我就持有股票了。
```

如果`buy`，就要从利润中减去`prices[i]`，如果`sell`，就要给利润增加`prices[i]`。今天的最大利润就是这两种可能选择中较大的那个。
而且注意 k 的限制，我们在选择`buy`的时候，把`k`减小了`1`，也可以在`sell`的时候减`1`，一样的。

2. 定义base case。

可以从-1开始定义也可以从0开始定义。如果从-1开始的话，base case 可以是如下情况：

```
dp[-1][k][0] = 0
解释：因为 i 是从 0 开始的，所以 i = -1 意味着还没有开始，这时候的利润当然是 0 。
dp[-1][k][1] = -infinity
解释：还没开始的时候，是不可能持有股票的，用负无穷表示这种不可能。
dp[i][0][0] = 0
解释：因为 k 是从 1 开始的，所以 k = 0 意味着根本不允许交易，这时候利润当然是 0 。
dp[i][0][1] = -infinity
解释：不允许交易的情况下，是不可能持有股票的，用负无穷表示这种不可能。
```

总结一下：

```
base case：
dp[-1][k][0] = dp[i][0][0] = 0
dp[-1][k][1] = dp[i][0][1] = -infinity

状态转移方程：
dp[i][k][0] = max(dp[i-1][k][0], dp[i-1][k][1] + prices[i])
dp[i][k][1] = max(dp[i-1][k][1], dp[i-1][k-1][0] - prices[i])
```

## 本题解析

如果直接把`k`代进去遍历的话，可能会出现超内存的错误，因为传入的`k`值会非常大，`dp`数组太大了。
而事实上，一次交易由买入和卖出构成，至少需要两天。所以说有效的限制 k 应该不超过 n/2，如果超过，就没有约束作用了，相当于股票II中`k`没有限制的情况。

## 参考代码

In [ ]:
class Solution:
    def maxProfit(self, k: int, prices):
        n = len(prices)
        # 若小于两天或允许交易次数小于1的话，不能交易
        if n < 2 or k <= 0:
            return 0
        # 当k>﴾len﴾prices﴿//2﴿时就相当于不限交易次数了，用股票2的代码解决更快
        if k >= n // 2:  
            ssum = 0
            for i in range(1, len(prices)):
                if prices[i] > prices[i-1]:
                    ssum += prices[i] - prices[i-1]
            return ssum
        # 其他情况
        dp = [[[0, 0] for _ in range(k+1)] for _ in range(n)]  # k这一维维数为k+1，因为要把k=0的给空出来
        # 定义base case
        for i in range(1, k+1):
            # 1. 第一天如果手里持有股票，肯定是第一天买入了
            dp[0][i][1] = -prices[0]
        for i in range(n):
            # 2. 当交易次数为0，不允许交易，不可能持有股票
            dp[i][0][1] = -float('inf')
        for i in range(1, n):
            for j in range(1, k+1):
                dp[i][j][0] = max(dp[i-1][j][0], dp[i-1][j][1] + prices[i])
                # 若要买入，应该考虑前一天(i-1)最高交易次数为(j-1)时的最大收益，因为本次买入要占用一次机会
                dp[i][j][1] = max(dp[i-1][j][1], dp[i-1][j-1][0] - prices[i])
        return dp[n-1][k][0]
    

## 总结

动态规划关键就在于列举出所有可能的「状态」，然后想想怎么穷举更新这些「状态」。一般用一个多维 dp 数组储存这些状态，从 base case 开始向后推进，推进到最后的状态，就是我们想要的答案。
